# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] GraphLab Create v1.8.3 started. Logging: C:\Users\Anirudh\AppData\Local\Temp\graphlab_server_1458707887.log.0


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Find what features had non-zero weight.

In [6]:
model_all.get('coefficients').print_rows(30)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
|  sqft_basement   |  None |

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [7]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [15]:
import numpy as np
l1 = np.logspace(1, 7, num=13)

for l1_penalty in l1 :
    model_l1_penalty = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty= l1_penalty,verbose = False)
    print model_l1_penalty.evaluate(validation), l1_penalty
    err = model_l1_penalty.predict(validation)- validation['price']
    rss = np.dot(err,err)
    print rss, l1_penalty

{'max_error': 4609435.483176051, 'rmse': 254847.40799944056} 10.0
6.25766285142e+14 10.0
{'max_error': 4609435.498283245, 'rmse': 254847.40804422507} 31.6227766017
6.25766285362e+14 31.6227766017
{'max_error': 4609435.54605639, 'rmse': 254847.40818584684} 100.0
6.25766286058e+14 100.0
{'max_error': 4609435.697128328, 'rmse': 254847.40863369455} 316.227766017
6.25766288257e+14 316.227766017
{'max_error': 4609436.17485978, 'rmse': 254847.41004992134} 1000.0
6.25766295212e+14 1000.0
{'max_error': 4609437.685579199, 'rmse': 254847.41452849974} 3162.27766017
6.25766317206e+14 3162.27766017
{'max_error': 4609442.46289368, 'rmse': 254847.428691777} 10000.0
6.25766386761e+14 10000.0
{'max_error': 4609457.570088896, 'rmse': 254847.47348767103} 31622.7766017
6.25766606749e+14 31622.7766017
{'max_error': 4609505.34323268, 'rmse': 254847.61522149367} 100000.0
6.25767302792e+14 100000.0
{'max_error': 4609656.415190306, 'rmse': 254848.06419097545} 316227.766017
6.25769507644e+14 316227.766017
{'max_

In [16]:
model_10 = graphlab.linear_regression.create(training, target='price',features=all_features,
                                            validation_set = None,
                                            l2_penalty = 0, l1_penalty= 10)

Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000003  | 0.607404     | 5737572.318059     | 289374.784200 |

| 6         | 7        | 0.000003  | 0.648431     | 5655864.782013     | 285354.826809 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [18]:
model_10.get('coefficients').print_rows(20)

+------------------+-------+------------------+--------+
|       name       | index |      value       | stderr |
+------------------+-------+------------------+--------+
|   (intercept)    |  None |  18993.4272128   |  None  |
|     bedrooms     |  None |  7936.96767903   |  None  |
| bedrooms_square  |  None |  936.993368193   |  None  |
|    bathrooms     |  None |  25409.5889341   |  None  |
|   sqft_living    |  None |  39.1151363797   |  None  |
| sqft_living_sqrt |  None |  1124.65021281   |  None  |
|     sqft_lot     |  None | 0.00348361822299 |  None  |
|  sqft_lot_sqrt   |  None |  148.258391011   |  None  |
|      floors      |  None |   21204.335467   |  None  |
|  floors_square   |  None |  12915.5243361   |  None  |
|    waterfront    |  None |  601905.594545   |  None  |
|       view       |  None |  93312.8573119   |  None  |
|    condition     |  None |  6609.03571245   |  None  |
|      grade       |  None |  6206.93999188   |  None  |
|    sqft_above    |  None |  4

*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [69]:
model_100000 = graphlab.linear_regression.create(training, target='price',features=all_features,
                                            validation_set = None,
                                            l2_penalty = 0, l1_penalty= 3614000000,verbose=False)
model_100000.get('coefficients').print_rows(20)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 230976.806285 |  None  |
|     bedrooms     |  None | 1.14560099083 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 14933.6714032 |  None  |
|   sqft_living    |  None | 31.5656056131 |  None  |
| sqft_living_sqrt |  None | 644.833260617 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 2603.67277918 |  None  |
|    sqft_above    |  None | 28.8318235462 |  None  |
|  sqft_basement   |  None |

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [70]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [71]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [74]:
for l1_penalty in l1_penalty_values:
    model_l1_penalty = graphlab.linear_regression.create(training, target='price', features=all_features,
                                                        validation_set = None, 
                                                        l2_penalty=0, l1_penalty= l1_penalty, verbose = False)
    print model_l1_penalty['coefficients']['value'].nnz(), l1_penalty

18 100000000.0
18 127427498.57
18 162377673.919
18 206913808.111
17 263665089.873
17 335981828.628
17 428133239.872
17 545559478.117
17 695192796.178
16 885866790.41
15 1128837891.68
15 1438449888.29
13 1832980710.83
12 2335721469.09
10 2976351441.63
6 3792690190.73
5 4832930238.57
3 6158482110.66
1 7847599703.51
1 10000000000.0


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [76]:
l1_penalty_min = 3414000000
l1_penalty_max = 3614000000

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [77]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [79]:
for l1_penalty in l1_penalty_values:
    model_l1_penalty = graphlab.linear_regression.create(training, target='price', features=all_features,
                                                        validation_set = None, 
                                                        l2_penalty=0, l1_penalty= l1_penalty, verbose = False)
    print model_l1_penalty.evaluate(validation), l1_penalty

{'max_error': 6104333.645499865, 'rmse': 329190.9362908268} 3414000000.0
{'max_error': 6105605.150091447, 'rmse': 329323.9103052535} 3424526315.79
{'max_error': 6106876.65468303, 'rmse': 329456.98542500316} 3435052631.58
{'max_error': 6108148.190197921, 'rmse': 329590.1647677116} 3445578947.37
{'max_error': 6109476.74921044, 'rmse': 329732.84014115407} 3456105263.16
{'max_error': 6110806.716852743, 'rmse': 329875.8630245816} 3466631578.95
{'max_error': 6112238.922736791, 'rmse': 330037.3817983885} 3477157894.74
{'max_error': 6113804.598174596, 'rmse': 330223.0301955972} 3487684210.53
{'max_error': 6115370.3116900185, 'rmse': 330408.89873183524} 3498210526.32
{'max_error': 6116935.987127824, 'rmse': 330594.97800291126} 3508736842.11
{'max_error': 6118501.700643246, 'rmse': 330781.27669652726} 3519263157.89
{'max_error': 6120067.376081052, 'rmse': 330967.7853883666} 3529789473.68
{'max_error': 6121631.720139391, 'rmse': 331154.1622965181} 3540315789.47
{'max_error': 6123185.470126945, 'r

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [81]:
model_l1_penalty = graphlab.linear_regression.create(training, target='price', features=all_features,
                                                        validation_set = None, 
                                                        l2_penalty=0, l1_penalty= 3414000000, verbose = False)
model_l1_penalty.get('coefficients').print_rows(20)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 220362.381399 |  None  |
|     bedrooms     |  None | 786.731241997 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 16049.4585874 |  None  |
|   sqft_living    |  None | 32.5610497478 |  None  |
| sqft_living_sqrt |  None | 698.522259956 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None |  2955.8268728 |  None  |
|    sqft_above    |  None | 30.2381415569 |  None  |
|  sqft_basement   |  None |